In [ ]:
import numpy as np
# test area
np.maximum.accumulate([1,2,3,1,5])
np.power(2,3)

In [ ]:
d.shape

# Load Data

In [ ]:
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt

base = 10
threshold = 0.6

df = pandas.read_csv('d.csv', index_col=0)
# df = df.filter(regex='^[^\u4e00-\u9fa5]+$', axis=0)
a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns = range(df.shape[1] - 1)
r = (b - a) / a
p = 1 / np.linalg.norm(r, axis=1)
r = (r.T * p).T.values
l = r.shape[0]

correlation = np.ones([l, l])
for i in range(l):
    for j in range(i + 1, l):
        s = (r[i] * r[j]).sum()
        correlation[i][j] = s
        correlation[j][i] = s
correlation[np.abs(correlation) > threshold] = 0
correlation[np.logical_and(np.abs(correlation) <= threshold, correlation != 0)] = 1

d = np.concatenate((np.ones([r.shape[0], 1]) * base,
                        np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1)

In [ ]:
latest = np.load('lasttime.npy')
print(latest.shape)
print(latest[0:3])
print(latest[-3:])
print(list(p[latest[0][0]]))

for a in latest:
    if 876 in a[0]:
        print(a)
        idx = a[0]
        break

dict = {}
for a in latest:
    for b in a[0]:
        if b in dict:
            dict[b] += 1
        else:
            dict[b] = 1
x = []
for key in dict:
    x.append([key, dict[key]])
x = sorted(x, key=lambda m: m[1], reverse=True)

for y in x[:500]:
    print(df.index[y[0]], y[0], y[1])

for a in latest[0:10]:
    print(list(df.index[a[0]]), a)

idx = latest[0][0]

# Test By tiger data

In [ ]:
columns = list(np.array(df.index)[idx])
print(columns)

dates = df.columns.astype(np.float32)
startDate = dates[0]
dates = np.round((dates - startDate) / 3600 / 24 / 1000 / 7) + 1

coridx = np.zeros([len(idx), len(idx)])
for i in range(len(idx)):
    for j in range(i):
        coridx[i][j] = correlation[idx[i]][idx[j]]
        coridx[j][i] = correlation[idx[j]][idx[i]]
print(coridx)

# basic info
a = d[idx][:, 1:]
b = d[idx][:, :-1]
r = a - b
contri= np.abs(r).mean(axis=1)
contri = contri / contri.min()
print("contribution: {}".format(contri))

graph = pandas.DataFrame(d[idx]).T
graph.columns = columns 
graph.plot()


d[idx].sum(axis=0)[-1]

s = (d[idx]).sum(axis=0)
print("percentage of total value: {}".format(p[idx] / p[idx].sum() * 100))

def evaluate(s, dates, components):
    # regression 
    s = s / len(idx)
    A, B = np.polyfit(dates, np.log(s), 1)
    print('A,B', A, B)
    y = np.exp(A * dates + B)
    last = s[0] < y[0]
    reg = 0
    for i in range(1, len(s)):
        if (s[i] < y[i]) != last:
            reg += 1
            last = s[i] < y[i]
    print('revisit times: ', reg)


    # max drawback 
    drop = 1 - s / np.maximum.accumulate(s)
    drop_i = np.argsort(drop)
    def getIJ(ij, i):
        return ij[i], np.argmax(s[:ij[i]])

    i, j = getIJ(drop_i, -1)     
    print('max drawback {:0.2f}% {:0.2f} {:0.2f} {} {}'.format(drop[i] * 100, s[i], s[j], i, j))
    print('average drop', drop.mean())
    pandas.DataFrame(drop).hist(bins=20)


    print(s[-1])
    print(dates[-1])

    plt.figure()
    plt.plot(dates, y)
    plt.plot(dates, s)
    plt.plot([i+1,j+1], [s[i], s[j]], 'o', color='Red', markersize=4)

    plt.figure(figsize=(16,9))
    plt.plot(dates, y)
    plt.plot(dates, s)
    if components is not None:
        plots = []
        for x in range(len(components.values.T)):
            plots.append(plt.plot(dates, components.values.T[x], label=components.columns[x])[0])
        plt.legend(handles=plots)
    plt.plot([i+1,j+1], [s[i], s[j]], 'o', color='Red', markersize=4)

    i2, j2 = getIJ(drop_i, -2)
    plt.plot([i2+1,j2+1], [s[i2], s[j2]], 'o', color='Blue', markersize=4)

evaluate(s, dates, graph)

In [ ]:
columns = list(np.array(df.index)[idx])
print(columns)
names = list(map(lambda c: c.split('#')[1], columns))

df2 = None
for n in names:
    f = '/Users/junnwang/Downloads/csv 2/' + n + '_week.csv'
    if df2 is None:
        df2 = pandas.read_csv(f, index_col=0).T
    else:
        df2 = pandas.concat([df2,pandas.read_csv(f, index_col=0).T], axis=1)
print(df2.dropna(axis=0,how='any').astype('float32').shape)     
df2 = df2.iloc[-160:,:]
d2 = df2.dropna(axis=1,how='any').astype('float32').T
a = d2.iloc[:, :-1]
b = d2.iloc[:, 1:]
print(a.shape)
print(b.shape)
b.columns = a.columns = range(d2.shape[1] - 1)
r = (b - a) / a
p2 = p[idx]
r = (r.T * p2).T.values


dates = d2.columns.astype(np.float32)
startDate = dates[0]
dates = np.round((dates - startDate) / 3600 / 24 / 1000 / 7) + 1

dd2 = np.concatenate((np.ones([r.shape[0], 1]) * base,
                        np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1)

s = dd2.sum(axis=0)
print("percentage of total value: {}".format(p2 / p2.sum() * 100))

dd2 = pandas.DataFrame(dd2.T)
dd2.columns = columns
evaluate(s, dates, dd2)

# Test by yahoo data

In [ ]:
from glob import glob
df = None
for f in glob('sss\\*.csv'):
    name = f.split('\\')[1].split('.')[0]
    if df is None:
        df = pandas.read_csv(f, index_col=0)[['Adj Close']]
        df.columns = [name]
    else:
        csv = pandas.read_csv(f, index_col=0)[['Adj Close']]
        csv.columns = [name]
        df = pandas.concat([df,csv], axis=1)
        
df = df.dropna().T
print(df.shape)
a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns
r = (((b - a) / a).T * p[idx] ).T
r = np.concatenate((np.ones([r.shape[0], 1]) * base,
                    np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1).T
r = pandas.DataFrame(r)
r.index = df.columns.astype(np.datetime64)
r.columns = df.index
r.plot(figsize=(16, 9))
print(r.shape)

y2 = r.sum(axis=1)
#plt.figure(figsize=(16, 9))
plt.plot(df.columns.astype(np.datetime64), y2 / len(idx))


# Test random output distribution

In [ ]:
import random
import dao

dist = []
n = 6
for i in range(500000):
    choice = random.sample(range(d.shape[0]), n)
    bad = True
    while bad:
        bad = False
        for k in range(len(choice)):
            for l in range(k):
                if np.abs(correlation[choice[k]][choice[l]]) == 0:
                    bad = True
                    break
            if bad:
                break
        if bad:
            choice = random.sample(range(d.shape[0]), n)

    result = dao.test_choice(choice)
    dist.append(result[1])
    
pandas.DataFrame(dist).hist(bins=1000)
pandas.qcut(dist,  [0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.999, 1])

# Load Weekly data and clean into d.csv

In [ ]:
%matplotlib inline
import pandas
from glob import glob
import numpy as np

df = None
for f in glob('D:\\work\\main\\IB\\csv\\*_week.csv'):
    if df is None:
        df = pandas.read_csv(f, index_col=0).T
    else:
        df = pandas.concat([df,pandas.read_csv(f, index_col=0).T], axis=1)

d = df.iloc[-112:-1,:]
d = d.dropna(axis=1,how='any').astype('float32').T
print(d.shape)
d.to_csv('d2.csv')

# Load Daily data

In [ ]:
from glob import glob
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt

daily = None
for f in glob('D:\\work\\main\\IB\\*_day.csv'):
    if daily is None:
        daily = pandas.read_csv(f, index_col=0).T
    else:
        daily = pandas.concat([daily,pandas.read_csv(f, index_col=0).T], axis=1)
daily.to_csv('day.csv')

# Test by day data

In [ ]:
from glob import glob
daily = pandas.read_csv('day.csv', index_col=0)[columns].T
print(columns)
moneyNeeded = 100 / (p[idx]/p[idx].sum()/daily.T.iloc[-1,:].values)[2]
tobuy = np.round(moneyNeeded * p[idx]/p[idx].sum() /  daily.T.iloc[-1,:].values)
moneyNeeded = tobuy * daily.T.iloc[-1,:].values
print('shares ', str(tobuy), tobuy.sum(), moneyNeeded)
moneyNeeded = moneyNeeded.sum()
print(moneyNeeded)

a = daily.iloc[:, :-1]
b = daily.iloc[:, 1:]
a.columns = b.columns
r = ((b / a - 1).T * p[idx] ).T
print(r.shape)

dates = daily.columns.astype(np.float32)
dates = np.round((dates - startDate) / 3600 / 24 / 1000) / 7 + 1
base = 10.4
r = np.concatenate((np.ones([r.shape[0], 1]) * base,
                    np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1).T
r = pandas.DataFrame(r)
r.columns = columns
r.index = dates
r.plot(figsize=(16,9))

y = np.exp(A * dates + B) * moneyNeeded / 10
y2 = r.sum(axis=1) / len(columns) * moneyNeeded / 10
plt.figure(figsize=(16,9))
plt.plot(dates, y)
plt.plot(dates, y2)
#plt.plot(111, np.exp(A * 111 + B), marker='o', color='r', ls='')
#y2[-1] / y[-1] - 1